In [3]:
import json
import re
import string
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from rouge_score import rouge_scorer
from transformers import RobertaTokenizerFast
from langchain_openai import ChatOpenAI
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [5]:
data = pd.read_csv('../data/ellipsis corpus.csv')
elipsis = [re.sub(r' _', '', i) for i in data['sentence']]
elipsis = [re.sub(r'_', '', i) for i in elipsis]
elipsis = [re.sub(r'\n', ' ', i) for i in elipsis]
answers_golden = data['suggested ellipsis resolution']

In [6]:
data

,sentence,suggested ellipsis resolution,ellipsis type,source
0,Работа с двухбайтовыми наборами символов — про...,состоит,gapping,https://github.com/dialogue-evaluation/AGRR-2019
1,"Выворачивает наизнанку, запад превращает в юг,...",превращает,gapping,https://github.com/dialogue-evaluation/AGRR-2019
2,Текущие обязательства компании в 2009 г. сокра...,уменьшилась,gapping,https://github.com/dialogue-evaluation/AGRR-2019
3,Первая конвенция была ратифицирована 80 процен...,была ратифицирована,gapping,https://github.com/dialogue-evaluation/AGRR-2019
4,"Но Христианин сказал Упрямому: — Нет, сосед, л...",мы получим,gapping,https://github.com/dialogue-evaluation/AGRR-2019
...,...,...,...,...
621,Что упало со стола? – Тарелка __.,упала со стола,answer ellipsis,original
622,Кто принёс эти документы? – Секретарь директор...,принёс эти документы,answer ellipsis,original
623,Когда откроют новый магазин? – В следующем мес...,откроют новый магазин,answer ellipsis,original
624,Что подарят директору на юбилей? __ Красивый е...,директору на юбилей подарят,answer ellipsis,original


In [ ]:
start = []
with_ = []
final = []
model = ChatOpenAI(model="gpt-4o-mini",
                   api_key="",
                   base_url="",
                   temperature=0)
for text in tqdm(elipsis):
    ans = model.invoke(f'''Дано предложение {text}. Оно содержит эллипсис, в нем пропущена часть информации.
    Постарайся восполнить как можно больше информации, не придумывай и не добавляй того, чего нет в контексте.
    Определи, 1) в каком месте пропущена информация, обозначь это место нижним подчеркиванием. 2) Восполни информацию и
    3) напиши новое предложение с восполненой информацией.
    Ответ дай в формате: изначальное - ответ на 1, эллипсис - ответ на 2, полное - ответ на 3. Ответ должен быть в формате json.''').content

    start.append(json.loads(ans.split('json')[1].strip('```'))['изначальное'])
    with_.append(json.loads(ans.split('json')[1].strip('```'))['эллипсис'])
    final.append(json.loads(ans.split('json')[1].strip('```'))['полное'])

  0%|          | 0/549 [00:00<?, ?it/s]

In [ ]:
model_answers_dict = {'initial': start, 'ellipsis': with_, 'final': final}
model_answers_df = pd.DataFrame.from_dict(model_answers_dict)

In [ ]:
model_answers_df

,initial,ellipsis,final
0,Работа с двухбайтовыми наборами символов — про...,что приводит к путанице и ошибкам при обработк...,Работа с двухбайтовыми наборами символов — про...
1,"Выворачивает наизнанку, запад превращает в юг,...",и зло — в добро,"Выворачивает наизнанку, запад превращает в юг,..."
2,Текущие обязательства компании в 2009 г. сокра...,долговые обязательства компании,Текущие обязательства компании в 2009 г. сокра...
3,Первая конвенция была ратифицирована 80 процен...,конвенция,Первая конвенция была ратифицирована 80 процен...
4,"Но Христианин сказал Упрямому: — Нет, сосед, л...","то, о чем я сейчас говорил, а именно — возможн...","Но Христианин сказал Упрямому: — Нет, сосед, л..."
...,...,...,...
544,Как часто он ходит в спортзал?,Он ходит в спортзал три раза в неделю.,Как часто он ходит в спортзал? Он ходит в спор...
545,На чём ты добираешься до работы?,На автобусе.,На чём ты добираешься до работы? Я добираюсь н...
546,Сколько чашек кофе бариста выпивает за день? П...,бариста выпивает за день,Сколько чашек кофе бариста выпивает за день? П...
547,До какого часа Петя будет в офисе завтра? До с...,Петя будет в офисе завтра до семи часов.,До какого часа Петя будет в офисе завтра? Петя...


In [ ]:
data['model_answer'] = with_

In [ ]:
data

,sentence,suggested ellipsis resolution,ellipsis type,source,model_answer
0,Работа с двухбайтовыми наборами символов — про...,состоит,gapping,https://github.com/dialogue-evaluation/AGRR-2019,что приводит к путанице и ошибкам при обработк...
1,"Выворачивает наизнанку, запад превращает в юг,...",превращает,gapping,https://github.com/dialogue-evaluation/AGRR-2019,и зло — в добро
2,Текущие обязательства компании в 2009 г. сокра...,уменьшилась,gapping,https://github.com/dialogue-evaluation/AGRR-2019,долговые обязательства компании
3,Первая конвенция была ратифицирована 80 процен...,была ратифицирована,gapping,https://github.com/dialogue-evaluation/AGRR-2019,конвенция
4,"Но Христианин сказал Упрямому: — Нет, сосед, л...",мы получим,gapping,https://github.com/dialogue-evaluation/AGRR-2019,"то, о чем я сейчас говорил, а именно — возможн..."
...,...,...,...,...,...
544,Как часто он ходит в спортзал? __ Три раза в н...,он ходит в спортзал,answer ellipsis,original,Он ходит в спортзал три раза в неделю.
545,На чём ты добираешься до работы? __ На автобусе.,я добираюсь до работы,answer ellipsis,original,На автобусе.
546,Сколько чашек кофе бариста выпивает за день? _...,бариста выпивает за день,answer ellipsis,original,бариста выпивает за день
547,До какого часа Петя будет в офисе завтра? __ Д...,Петя будет в офисе завтра,answer ellipsis,original,Петя будет в офисе завтра до семи часов.


In [ ]:
data.to_csv('final_table.csv', index=False)

In [ ]:
puctuation_to_clean = "!\"#$%&'()*+,—./:;<=>?@[\]^_`{|}~"

In [ ]:
model_answers_clean = []
golden_answers_clean = []
for i in range(len(data['model_answer'])):
    model_answers_clean.append(re.sub(r'ё', r'е', data['model_answer'][i].translate(str.maketrans('', '', puctuation_to_clean)).lower()))
    golden_answers_clean.append(re.sub(r'ё', r'е', data['suggested ellipsis resolution'][i].translate(str.maketrans('', '', string.punctuation)).lower()))

In [ ]:
model_answers_clean

['что приводит к путанице и ошибкам при обработке данных',
 'и зло  в добро',
 'долговые обязательства компании',
 'конвенция',
 'то о чем я сейчас говорил а именно  возможность достичь успеха и признания',
 'срок действия контрактов',
 'и это вызывает недоумение у многих людей',
 'в состоянии когда они находятся в растворе или расплавленном виде',
 'различным кожным заболеваниям',
 'группа ворвалась',
 'котировки акций',
 'по сравнению с первоначальным уровнем расходов',
 'от структуры данных которые используются в запросе',
 'в активах  458',
 'я откликнусь на дурака ты откликнёшься на дуру',
 'от неправильных поступков и трудностей в жизни',
 'некоторые из них были наемными мирянами а некоторые жрецами',
 'котировки на нефть',
 'как и прежде',
 'жизнь природы',
 'и его свойства',
 'не любит',
 'аг и аг',
 'включая но не ограничиваясь соединенными штатами европейским союзом и другими юрисдикциями',
 'отдать честь своему командиру который находился рядом',
 'доля его активов в собстве

In [ ]:
for_metrics = []
for i in range(len(model_answers_clean)):
    if model_answers_clean[i] == golden_answers_clean[i]:
        for_metrics.append(1)
    else:
        for_metrics.append(0)

print(sum(for_metrics)/len(for_metrics))

golden = [1] * len(data['model_answer'])
print('accuracy: ', accuracy_score(golden, for_metrics))
print('recall_score: ', recall_score(golden, for_metrics))
print('f1_score: ', f1_score(golden, for_metrics))
print('precision_score: ', precision_score(golden, for_metrics))

0.20218579234972678
accuracy:  0.20218579234972678
recall_score:  0.20218579234972678
f1_score:  0.33636363636363636
precision_score:  1.0


In [ ]:
for_metrics_set = []
for i in range(len(model_answers_clean)):
    if set(model_answers_clean[i]) == set(golden_answers_clean[i]):
        for_metrics_set.append(1)
    else:
        for_metrics_set.append(0)

print(sum(for_metrics_set)/len(for_metrics_set))

golden = [1] * len(data['model_answer'])
print('accuracy: ', accuracy_score(golden, for_metrics_set))
print('recall_score: ', recall_score(golden, for_metrics_set))
print('f1_score: ', f1_score(golden, for_metrics_set))
print('precision_score: ', precision_score(golden, for_metrics_set))

0.21493624772313297
accuracy:  0.21493624772313297
recall_score:  0.21493624772313297
f1_score:  0.3538230884557721
precision_score:  1.0


In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained('blinoff/roberta-base-russian-v0', max_len=512)

r_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], tokenizer=tokenizer)

In [ ]:
rouge1_precision = []
rouge1_recall = []
rouge1_f = []
rouge2_precision = []
rouge2_recall = []
rouge2_f = []
rougeL_precision = []
rougeL_recall = []
rougeL_f = []
for sentence_pair in range(len(model_answers_clean)):
  sent_pair_r_score = r_scorer.score(model_answers_clean[sentence_pair], golden_answers_clean[sentence_pair])
  rouge1_precision.append(sent_pair_r_score['rouge1'][0])
  rouge1_recall.append(sent_pair_r_score['rouge1'][1])
  rouge1_f.append(sent_pair_r_score['rouge1'][2])
  rouge2_precision.append(sent_pair_r_score['rouge2'][0])
  rouge2_recall.append(sent_pair_r_score['rouge2'][1])
  rouge2_f.append(sent_pair_r_score['rouge2'][2])
  rougeL_precision.append(sent_pair_r_score['rougeL'][0])
  rougeL_recall.append(sent_pair_r_score['rougeL'][1])
  rougeL_f.append(sent_pair_r_score['rougeL'][2])

In [ ]:
data['rouge1_precision'] = rouge1_precision
data['rouge1_recall'] = rouge1_recall
data['rouge1_f'] = rouge1_f
data['rouge2_precision'] = rouge2_precision
data['rouge2_recall'] = rouge2_recall
data['rouge2_f'] = rouge2_f
data['rougeL_precision'] = rougeL_precision
data['rougeL_recall'] = rougeL_recall
data['rougeL_f'] = rougeL_f

In [ ]:
data

,sentence,suggested ellipsis resolution,ellipsis type,source,model_answer,rouge1_precision,rouge1_recall,rouge1_f,rouge2_precision,rouge2_recall,rouge2_f,rougeL_precision,rougeL_recall,rougeL_f
0,Работа с двухбайтовыми наборами символов — про...,состоит,gapping,https://github.com/dialogue-evaluation/AGRR-2019,что приводит к путанице и ошибкам при обработк...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
1,"Выворачивает наизнанку, запад превращает в юг,...",превращает,gapping,https://github.com/dialogue-evaluation/AGRR-2019,и зло — в добро,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
2,Текущие обязательства компании в 2009 г. сокра...,уменьшилась,gapping,https://github.com/dialogue-evaluation/AGRR-2019,долговые обязательства компании,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
3,Первая конвенция была ратифицирована 80 процен...,была ратифицирована,gapping,https://github.com/dialogue-evaluation/AGRR-2019,конвенция,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
4,"Но Христианин сказал Упрямому: — Нет, сосед, л...",мы получим,gapping,https://github.com/dialogue-evaluation/AGRR-2019,"то, о чем я сейчас говорил, а именно — возможн...",0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
544,Как часто он ходит в спортзал? __ Три раза в н...,он ходит в спортзал,answer ellipsis,original,Он ходит в спортзал три раза в неделю.,1.0,0.555556,0.714286,1.0,0.500000,0.666667,1.0,0.555556,0.714286
545,На чём ты добираешься до работы? __ На автобусе.,я добираюсь до работы,answer ellipsis,original,На автобусе.,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
546,Сколько чашек кофе бариста выпивает за день? _...,бариста выпивает за день,answer ellipsis,original,бариста выпивает за день,1.0,1.000000,1.000000,1.0,1.000000,1.000000,1.0,1.000000,1.000000
547,До какого часа Петя будет в офисе завтра? __ Д...,Петя будет в офисе завтра,answer ellipsis,original,Петя будет в офисе завтра до семи часов.,1.0,0.625000,0.769231,1.0,0.571429,0.727273,1.0,0.625000,0.769231


In [ ]:
print('rouge1_precision: ', data['rouge1_precision'].mean())
print('rouge1_recall: ', data['rouge1_recall'].mean())
print('rouge1_f: ', data['rouge1_f'].mean())
print('rouge2_precision: ', data['rouge2_precision'].mean())
print('rouge2_recall: ', data['rouge2_recall'].mean())
print('rouge2_f: ', data['rouge2_f'].mean())
print('rougeL_precision: ', data['rougeL_precision'].mean())
print('rougeL_recall: ', data['rougeL_recall'].mean())
print('rougeL_f: ', data['rougeL_f'].mean())

rouge1_precision:  0.4231452012872778
rouge1_recall:  0.35189667072859687
rouge1_f:  0.3623587667300963
rouge2_precision:  0.31293488647455103
rouge2_recall:  0.2696790060813992
rouge2_f:  0.27453870445427636
rougeL_precision:  0.41953257226481266
rougeL_recall:  0.3496558855723505
rougeL_f:  0.35978843683736206


In [ ]:
rouge1_precision:  0.4174429143008378
rouge1_recall:  0.3496525668576354
rouge1_f:  0.3601388745401865
rouge2_precision:  0.31075647182204563
rouge2_recall:  0.2692751133766884
rouge2_f:  0.27286094181709974
rougeL_precision:  0.41480174854491797
rougeL_recall:  0.34815906113719297
rougeL_f:  0.35837809736957327

In [ ]:
count_df_all = data['ellipsis type'].value_counts().reset_index()
count_df_all.columns = ['value_all', 'count_all']
count_df_all

,value_all,count_all
0,sluicing,100
1,polarity_ellipsis,99
2,NP_ellipsis,99
3,gapping,97
4,VP_ellipsis,96
5,answer ellipsis,32
6,stripping,14
7,verb-stranding,3
8,"NP-ellipsis, gapping",1
9,"NP_ellipsis, VP_ellipsis",1


In [ ]:
count_df_good = data[data['rougeL_f'] > 0.35]['ellipsis type'].value_counts().reset_index()
count_df_good.columns = ['value_good', 'count_good']
count_df_good

,value_good,count_good
0,VP_ellipsis,62
1,polarity_ellipsis,54
2,NP_ellipsis,53
3,sluicing,18
4,answer ellipsis,13
5,stripping,10
6,gapping,10
7,verb-stranding,3
8,PP_ellipsis,1
9,"gapping, PP-ellipsis",1


In [ ]:
count_df_bad = data[data['rougeL_f'] < 0.2]['ellipsis type'].value_counts().reset_index()
count_df_bad.columns = ['value_bad', 'count_bad']
count_df_bad

,value_bad,count_bad
0,gapping,83
1,sluicing,65
2,NP_ellipsis,43
3,polarity_ellipsis,34
4,VP_ellipsis,26
5,answer ellipsis,15
6,stripping,2
7,"gapping, NP-ellipsis",1
8,"NP-ellipsis, gapping",1
9,"NP_ellipsis, VP_ellipsis",1


In [ ]:
merged = pd.merge(pd.merge(count_df_all, count_df_good, how='left', left_on='value_all', right_on='value_good'),
                  count_df_bad, how='left', left_on='value_all', right_on='value')

KeyError: 'value'

In [ ]:
merged['good_percentage'] = (np.divide(merged['count_good'],
                        merged['count_all'],
                        where=(merged['count_all']!=0|merged['count_all']!=None))).round(2)
merged['good_percentage'] = (np.divide(merged['count_bad'],
                        merged['count_all'],
                        where=(merged['count_all']!=0|merged['count_all']!=None))).round(2)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
merged['good_percentage'] = (merged['count_good'] / merged['count_all'] * 100).round(2)
merged['bad_percentage'] = (merged['count_bad'] / merged['count_all'] * 100).round(2)

In [ ]:
merged[merged['count_all'] > 50].sort_values('good_percentage', ascending=False)

,value_all,count_all,value_good,count_good,value_bad,count_bad,good_percentage,bad_percentage
4,VP_ellipsis,96,VP_ellipsis,62.0,VP_ellipsis,26.0,64.58,27.08
1,polarity_ellipsis,99,polarity_ellipsis,54.0,polarity_ellipsis,34.0,54.55,34.34
2,NP_ellipsis,99,NP_ellipsis,53.0,NP_ellipsis,43.0,53.54,43.43
0,sluicing,100,sluicing,18.0,sluicing,65.0,18.00,65.00
3,gapping,97,gapping,10.0,gapping,83.0,10.31,85.57


In [ ]:
merged[merged['count_all'] > 50].sort_values('bad_percentage', ascending=False)

,value_all,count_all,value_good,count_good,value_bad,count_bad,good_percentage,bad_percentage
3,gapping,97,gapping,10.0,gapping,83.0,10.31,85.57
0,sluicing,100,sluicing,18.0,sluicing,65.0,18.00,65.00
2,NP_ellipsis,99,NP_ellipsis,53.0,NP_ellipsis,43.0,53.54,43.43
1,polarity_ellipsis,99,polarity_ellipsis,54.0,polarity_ellipsis,34.0,54.55,34.34
4,VP_ellipsis,96,VP_ellipsis,62.0,VP_ellipsis,26.0,64.58,27.08
